In [ ]:
import numpy as np
from sklearn.svm import SVC
import pickle
from sklearn.metrics import classification_report
import keras

In [77]:
DS_CDFV1 = 'celeb_df_v1/'
DS_CDFV2 = 'celeb_df_v2/'

DS_ORGINAL = 'dataset_original/'
DS_SPLIT = 'dataset_split/'
DS_IFRAMES = 'dataset_iframes/'
DS_FACE = 'dataset_face/'
DS_FACE_IMG = 'dataset_face_img/'
DS_SRM_SNIPPETS = 'dataset_srm_snippets_5/'
DS_SEGMENTS = 'dataset_segments/'
DS_RAW = 'dataset_raw/'
DS_RESIDUALS = 'dataset_residuals/'
DS_TEMPORAL = 'dataset_temporal/'
MODELS = 'models/'


SEG_1 = 'seg_1/'
SEG_2 = 'seg_2/'
SEG_3 = 'seg_3/'
SEG_4 = 'seg_4/'
SEG_5 = 'seg_5/'

SEG = ['seg_1_', 'seg_2_', 'seg_3_', 'seg_4_', 'seg_5_']

DS_TRAIN = 'train_dataset/'
DS_TEST = 'test_dataset/'
DS_VAL = 'val_dataset/'

CLASS_FAKE = 'fake/'
CLASS_REAL = 'real/'


TOP_LEVEL_1 = [DS_SPLIT, DS_IFRAMES, DS_FACE, DS_FACE_IMG, DS_SRM_SNIPPETS]
TOP_LEVEL_2 = [DS_SEGMENTS, DS_RAW, DS_RESIDUALS]
SEGMENTS = [SEG_1, SEG_2, SEG_3, SEG_4, SEG_5]
SPLIT = [DS_TRAIN, DS_TEST, DS_VAL]
CLASS = [CLASS_REAL, CLASS_FAKE]

DATASET = [DS_CDFV1, DS_CDFV2]

In [78]:
stream_predictions = []
actual_class = []
final_predictions = []

def create_dataset(src_dir, filename):
    file = open(src_dir + filename)

    for line in file:
        [pred1, pred2, pred3, actual] = np.float_(line.split())
        stream_predictions.append([pred1, pred2, pred3])
        actual_class.append(actual)

In [79]:
create_dataset(DS_CDFV1, 'sample_test_predictions.txt')

In [80]:
model = SVC(kernel='linear', random_state=0,  probability=True)  
model.fit(stream_predictions, actual_class) 

SVC(kernel='linear', probability=True, random_state=0)

In [88]:
final_predictions = model.predict(stream_predictions)
final_predictions

array([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.])

In [82]:
final_predictions = model.predict_proba(stream_predictions)
final_predictions

array([[0.09984447, 0.90015553],
       [0.89999988, 0.10000012],
       [0.09984447, 0.90015553],
       [0.89999988, 0.10000012],
       [0.09984447, 0.90015553],
       [0.89999988, 0.10000012],
       [0.09984447, 0.90015553],
       [0.89999988, 0.10000012],
       [0.09984447, 0.90015553],
       [0.89999988, 0.10000012],
       [0.09984447, 0.90015553],
       [0.89999988, 0.10000012],
       [0.09984447, 0.90015553],
       [0.89999988, 0.10000012],
       [0.09984447, 0.90015553],
       [0.89999988, 0.10000012]])

In [83]:
final_predictions = model.decision_function(stream_predictions)
final_predictions

array([ 1.00000001, -0.99999995,  1.00000001, -0.99999995,  1.00000001,
       -0.99999995,  1.00000001, -0.99999995,  1.00000001, -0.99999995,
        1.00000001, -0.99999995,  1.00000001, -0.99999995,  1.00000001,
       -0.99999995])

In [ ]:
with open(MODELS + '/SCORE_AGGREGATION/score_agg', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
with open(MODELS + '/SCORE_AGGREGATION/score_agg', 'rb') as f:
    model = pickle.load(f)

In [75]:
report = classification_report(actual_class, final_predictions)
print(report)

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         8
         1.0       1.00      1.00      1.00         8

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [89]:
def metrics(actual, predicted):
    m = keras.metrics.CategoricalAccuracy()
    m.update_state(actual, predicted)
    print(f'Categorical Accuracy - {m.result().numpy()}')
    
    m = keras.metrics.Precision()
    m.update_state(actual, predicted)
    print(f'Precision - {m.result().numpy()}')

    m = keras.metrics.Recall()
    m.update_state(actual, predicted)
    print(f'Recall - {m.result().numpy()}')

    m = keras.metrics.AUC()
    m.update_state(actual, predicted)
    print(f'AUC - {m.result().numpy()}')

In [90]:
metrics(actual_class, final_predictions)

Categorical Accuracy - 1.0
Precision - 1.0
Recall - 1.0
AUC - 1.0
